In [25]:
"""
Convert COCO RLE annotations to RedBrick AI mask format. 

Requires
-----------
pycocotools https://github.com/cocodataset/cocoapi
"""
import pycocotools.mask as mask_util
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 300
import numpy as np
import redbrick

In [26]:
# Standard SDK setup
api_key = "TODO"
org_id = "TODO"
project_id = "TODO"
url = "https://api.redbrickai.com"

project = redbrick.get_project(api_key, url, org_id, project_id)

In [20]:
# Read in your coco annotations
import json
labels = {}
coco_annotations_file = "TODO.json"
with open(coco_annotations_file, "r") as file:
    labels = json.load(file)

In [21]:
def convert_to_masks(labels):
    """
    Converts COCO annotations to 2D, single channel mask.

    Parameters
    -------------
    labels: dict
        Dictionary containing annotations in COCO format, with segmentation labels
        in RLE format. 
    
    Returns
    -------------
    id_to_mask: dict
        id_to_mask {image_id: mask}. Contains a mapping from image_id
        to the numpy mask. The mask is a numpy array of shape (m, n, 1), with
        array values equal to category_id. 
    """
    id_to_mask = {}
    for label in labels["annotations"]:
        mask = mask_util.decode(label["segmentation"])[:, :]
        mask *= label["category_id"]

        if label["image_id"] in id_to_mask:
            idxs = np.where(mask > 0)
            id_to_mask[label["image_id"]][idxs] = mask[idxs]
        else:
            id_to_mask[label["image_id"]] = mask
    
    return id_to_mask

def make_color_mask(mask, class_id_map, taxonomy):
    """
    Creates a color mask from a single channel mask. 

    Parameters
    ------------
    mask: np.ndarray (m,n,1)
        Single channel numpy mask
    
    class_id_map: dict
        Maps from {class_id: [r, g, b]}. Modifies in-place. 
    
    Returns
    ------------
    color_mask: np.ndarray (m, n, 3)
        Three channel RGB numpy mask
    """
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3))
    class_ids = np.unique(mask)  
    for i in class_ids:
        if i == 0:
            # don't add color to background
            continue
        indexes = np.where(mask == i)
        color = np.array(cm.tab10(int(i - 1)))[0:3] * 255
        color = color.astype(np.uint8)
        color_mask[indexes] = color

        # update class_id_map
        class_id_map[taxonomy[i]] = color

    return color_mask

def create_taxonomy(labels):
    """
    Creates taxonomy object from COCO annotations. 

    Parameters
    -------------
    labels: dict
        Dictionary containing COCO annotations. 
    
    Returns
    -------------
    taxonomy: dict
        Mapping from class_id to category_name. 
    """
    taxonomy = {}
    for category in labels["categories"]:
        taxonomy[category["id"]] = category["name"]
    
    return taxonomy


## Convert RLE annotations to masks, and save locally in RBAI format

In [27]:
from matplotlib import cm

# required for running in jupyter notebook
# https://docs.redbrickai.com/python-sdk/sdk-overview#running-inside-of-a-jupyter-notebook
import nest_asyncio
nest_asyncio.apply()

# convert to masks
id_to_mask = convert_to_masks(labels)

# create taxonomy object
taxonomy = create_taxonomy(labels)

# create datapoint_map file
datapoint_map = {}
for image in labels["images"]:
    mask_name = "%s.png" % image["id"]
    datapoint_map[mask_name] = image["file_name"]

# class_id_map object
class_id_map = {}

# save masks locally
for id in id_to_mask:
    mask = id_to_mask[id]
    color_mask = make_color_mask(mask, class_id_map, taxonomy)
    color_mask = color_mask.astype(np.uint8)
    project.upload.create_datapoint_from_masks(
        redbrick.StorageMethod.REDBRICK, color_mask, class_id_map, datapoint_map["%s.png" % id]
    )

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO]:  Creating datapoints


  0%|          | 0/1 [00:00<?, ?it/s]

[INFO]:  Creating datapoints


100%|██████████| 1/1 [00:01<00:00,  1.15s/it]
